In [7]:
from brfinance import CVMAsyncBackend
import pandas
from datetime import datetime, date
import requests
from bs4 import BeautifulSoup
import re

In [8]:
class CVM():

    def __init__(self) -> None:

        self.cvm_http_client = CVMAsyncBackend()

    def get_cvm_categories(self):
        """ EXTRAI AS INFORMAÇÕES DE CATEGORIAS
            I.E. OS TIPOS DE DOCUMENTOS DISPONÍVEIS

            UTILIZA O BrFINANCE E DADOS DA CVM
        """
        categories = self.cvm_http_client.get_consulta_externa_cvm_categories()
        categories = pandas.DataFrame.from_dict(categories, orient = "index").reset_index()
        categories.columns = ["BrFin", "Description"]
        categories.to_csv("CategoriasDocsCVM.txt", index = False)

    def get_cvm_codes(self):
        """ EXTRAI OS CODIGOS CVM + STATUS CVM

            UTILIZA A BIBLIOTECA BrFINANCE E DADOS DA CVM
        """
        codes = self.cvm_http_client.get_cvm_codes()
        codes = pandas.DataFrame.from_dict(codes, orient = "index")
        pattern = r'\((.*?)\)'
        codes[1] = codes[0].str.extract(pattern).fillna("-")
        codes[0] = codes[0].str.replace(pattern, "", regex = True)
        codes = codes.reset_index()
        codes.columns = ["CVMCode", "FirmName", "CVMStatus"]
        codes.to_csv("CodigosDocsCVM.txt", index = False)

    def firms_informations(self, extract_data:bool = True, fetch_data:bool = True, fetch_br_fin:bool = False, record_file:str = "EmpresasBrasilCVM"):

        """ ADQUIRE INFORMAÇÕES DE COMPANHIAS ABERTAS BRASILEIRAS VIA CVM

            REFERENCIA DE SITE: https://cvmweb.cvm.gov.br/SWB/Sistemas/SCW/CPublica/CiaAb/FormBuscaCiaAbOrdAlf.aspx?LetraInicial=3

            EXTRACT_DATA [BOOL] > EXTRAI OS DADOS EM TABELAS, SEM CABEÇALHO E CRIA ARQUIVO DE REFERENCIA DE DADOS EM FORMATO .TXT
            FETCH_DATA   [BOOL] > AJUSTA DATAFRAME DE REFERÊNCIA CVM
            FETCH_BR_FIN [BOOL] > VERIFICA AS INFORMAÇÕES PRESENTES NA LIB BR_FINANCE
        """
        if extract_data:
            # INFORMAÇÕES SOBRE AS COMPANHIAS BRASILEIRAS
            infos = []

            # TODAS AS LETRAS MAIUSCULAS E NUMEROS
            # CONFORME MOSTRADO NO BUSCADOR DA CVM
            capital_letters_and_numbers = []

            # TODAS AS LETRAS MAIUSCULAS VIA ASCII
            for ascii_value in range(ord('A'), ord('Z') + 1):
                capital_letters_and_numbers.append(chr(ascii_value))

            # TODOS OS DIGITOS POSSIVEIS
            for digit in range(10):
                capital_letters_and_numbers.append(str(digit))

            # PARA CADA POSSIBILIDADE NAS LISTAS CRIADAS
            # TENTAMOS ACESSAR AS TABELAS DISPONIVEIS
            for focus in capital_letters_and_numbers:

                # USA O SITE BASE DA CVM E ALTERA A LETRA INICIAL PARA BUSCAR AS TABELAS
                url = f'https://cvmweb.cvm.gov.br//SWB//Sistemas//SCW//CPublica//CiaAb//FormBuscaCiaAbOrdAlf.aspx?LetraInicial={focus}'
                print(f"Researching firms: Start with {focus}")

                # USA O REQUEST PARA ACESSAR A URL
                response = requests.get(url)

                # SE O STATUS CODE = 200, ACESSO FOI UM SUCESSO
                if response.status_code == 200:

                    try:

                        # REALIZA O PARSING DO HTML VIA BEAUTIFULSOUP
                        html_content = response.text
                        soup = BeautifulSoup(html_content, 'html.parser')

                        # TODAS AS TABELAS NO HTML
                        tables = soup.find_all('table')

                        for table in tables:

                            # PROCESSAMOS OS ELEMENTOS 'TABELA'
                            rows = table.find_all('tr')

                            for row in rows:

                                # CÉLULAS DA TABELA
                                cells = row.find_all(['th', 'td'])
                                # TEXTO COMPLETO DE INFORMAÇÕES A SER EXTRAÍDO
                                full_txt = ""

                                for cell in cells:

                                    # ADQUIRE O TEXTO, SEM ESPAÇOS
                                    txt = cell.get_text(strip = True)

                                    if not txt.startswith(("CNPJ", "Selecione", " ")):

                                        # NÃO DESEJAMOS OS CABEÇALHOS PADRÃO
                                        full_txt += txt + ";"

                                    else: break

                                # CASO O TEXTO SEJA VALIDO, ACRESCENTA O ARQUIVO EMPRESAS BRASIL
                                if len(full_txt) > 0:

                                    infos.append(full_txt)                     

                    except Exception as exc:

                        # NOS AJUDA A COMPREENDER SE HOUVE UM ERRO E PORQUE
                        print(f"[ERROR] at {focus}: {exc}")

                else:

                    # NÃO FOI POSSIVEL REALIZAR O REQUEST
                    print(f"[WARNING] at {focus}: Response Status - {response.status_code}")

            # CRIA O ARQUIVO FINAL DE INFORMAÇÕES
            with open(record_file + ".txt", "w+") as record:
                for info in infos:
                    try:
                        record.write(info + "\n")
                    except Exception as exc:
                        print(info, exc)

        if fetch_data:

            # MESMA REFERÊNCIA DE FORMATO UTILIZADA PELO BrFINANCE
            def format_number(number):
                return f'{number:06d}'

            # EXTRAI O DATAFRAME DE INFORMAÇÕES DA CVM
            df = pandas.read_csv(
                record_file + ".txt", sep = ";", encoding = "latin1", header = None
            ).dropna(axis = 1)
            # NOME PADRÃO DAS COLUNAS
            df.columns = ["CNPJ", "FIRM_NAME", "CVM_FIRM_TYPE", "CVM_CODE", "CVM_STATUS"]
            # CORRIGE OS CÓDIGOS
            df["CVM_CODE"] = df["CVM_CODE"].apply(format_number)
            # SOBSCREVE AS INFORMAÇÕES
            df.to_csv(record_file + ".txt", index = False)

        if fetch_br_fin:

            cvm_codes = self.cvm_http_client.get_cvm_codes()
            df = pandas.read_csv(record_file + ".txt")
            print(cvm_codes)
            print(df)

    def get_selected_firms(self):

        df = pandas.read_csv("feeder_db//Brazil.txt")
        df = df[(df.CNPJ != "-") & (df.CVM_CODE != "-")]
        self.firms = df.copy()
        display(df)
    
    def documents_cvm(self, start_date = date(1995,12,31)):

        CVM.get_selected_firms(self)

        categories = pandas.read_csv("feeder_db//CategoriasDocsCVM.txt")["BrFin"].tolist()

        for cvm_code in self.firms["CVM_CODE"].unique():

            got = False
            retries = 0

            while not got:

                try:

                    if retries < 3:

                        search_result = self.cvm_http_client.get_consulta_externa_cvm_results(
                            start_date = start_date,
                            end_date = date.today(),
                            cod_cvm = [cvm_code],
                            participant_type = [1],
                            category = categories,
                            last_ref_date = False
                        )

                        search_result.to_csv(f"cvm//Docs_CVM_{cvm_code}.txt")
                        print(f"Dados armazenados com sucesso - CVM {cvm_code}")

                    got = True

                except Exception as exc:

                    retries += 1
                    print(f"ERROR - Aquisicao de dados CVM - Code: {cvm_code} - Retry: {retries}")



In [9]:
cvm = CVM()
#cvm.get_cvm_categories()
#cvm.get_cvm_codes()
#cvm.firms_informations(extract_data = False, fetch_data = False, fetch_br_fin = True)
cvm.documents_cvm()

,NAME,CLASS,CODE,CNPJ,STOCK_TYPE,ISIN,SETOR(BOVESPA),SUBSETOR(BOVESPA),SEGMENTO(BOVESPA),CVM_CODE,CVM_STATUS,CVM_FIRM_TYPE,FIRM_NAME,SOURCE_ECONOMATICA,SOURCE_CVM,SOURCE_BRFIN
0,3R PETROLEUM,ON,RRRP3,12.091.809/0001-55,AÇÃO,BRRRRPACNOR5,PETRÓLEO GÁS E BIOCOMBUSTÍVEIS,PETRÓLEO GÁS E BIOCOMBUSTÍVEIS,EXPLORAÇÃO REFINO E DISTRIBUIÇÃO,025291,Concedido em 09/11/2020,CIAS ABERTAS,3R PETROLEUM OLÉO E GÁS S.A.,True,True,False
1,3TENTOS,ON,TTEN3,94.813.102/0001-70,AÇÃO,BRTTENACNOR0,CONSUMO NÃO CÍCLICO,AGROPECUÁRIA,AGRICULTURA,025950,Concedido em 21/06/2021,CIAS ABERTAS,TRÊS TENTOS AGROINDUSTRIAL S.A.,True,True,False
2,521 PARTICIP,ON,QVUM3B,01.547.749/0001-16,AÇÃO,-,-,-,-,016330,Concedido em 11/07/1997,CIAS ABERTAS,521 PARTICIPAÇOES S.A. - EM LIQUIDAÇÃO EXTRAJU...,True,True,False
3,524 PARTICIP,ON,QVQP3,01.851.771/0001-55,AÇÃO,BRQVQPACNOR1,OUTROS,OUTROS,OUTROS,016284,Concedido em 30/05/1997,CIAS ABERTAS,524 PARTICIPAÇOES SA,True,True,False
4,A P PARTICIPACOES,ON;PN,APPA3;APPA4,02.288.752/0001-25,AÇÃO,-,-,-,-,016802,Cancelado em 23/12/2004,CIAS ABERTAS,A.P. PARTICIPAÇOES SA,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897,YARA BRASIL,ON;PN,ILMD3;ILMD4,92.660.604/0001-82,AÇÃO,BRILMDACNOR4;BRILMDACNPR1,-,-,-,000140,Cancelado em 18/11/2011,CIAS ABERTAS,YARA BRASIL FERTILIZANTES,True,True,False
898,YBYRA S/A,ON,YBRA3,02.217.319/0001-07,AÇÃO,BRYBRAACNOR7,OUTROS,OUTROS,OUTROS,016705,Concedido em 08/12/1997,CIAS ABERTAS,YBYRA CAPITAL S.A,True,True,False
899,YDUQS PART,ON;PN;UNT N2,YDUQ3;ESTC4;ESTC11,08.807.432/0001-10,AÇÃO,BRYDUQACNOR3;-,CONSUMO CÍCLICO,DIVERSOS,SERVIÇOS EDUCACIONAIS,021016,Concedido em 26/07/2007,CIAS ABERTAS,YDUQS PARTICIPACOES S.A.,True,True,False
900,ZAIN PART,ON,OPZI3B,02.363.918/0001-20,AÇÃO,BROPZIACNOR4,FINANCEIRO,OUTROS,OUTROS,017361,Cancelado em 14/08/2020,CIAS ABERTAS,ZAIN PARTICIPAÇÕES S/A,True,True,False


Dados armazenados com sucesso - CVM 025291
Dados armazenados com sucesso - CVM 025950
Dados armazenados com sucesso - CVM 016330
Dados armazenados com sucesso - CVM 016284
Dados armazenados com sucesso - CVM 016802
Dados armazenados com sucesso - CVM 020958
ERROR - Aquisicao de dados CVM - Code: 020206 - Retry: 1
ERROR - Aquisicao de dados CVM - Code: 020206 - Retry: 2
ERROR - Aquisicao de dados CVM - Code: 020206 - Retry: 3
Dados armazenados com sucesso - CVM 005380
Dados armazenados com sucesso - CVM 000108
Dados armazenados com sucesso - CVM 021725
Dados armazenados com sucesso - CVM 025283
Dados armazenados com sucesso - CVM 025640
Dados armazenados com sucesso - CVM 019313
Dados armazenados com sucesso - CVM 016527
Dados armazenados com sucesso - CVM 018350
Dados armazenados com sucesso - CVM 026174
Dados armazenados com sucesso - CVM 018104
Dados armazenados com sucesso - CVM 020150
Dados armazenados com sucesso - CVM 022179
Dados armazenados com sucesso - CVM 019437
Dados armaze